In [1]:
import os
import keras

import numpy as np
import tensorflow as tf
from tqdm import tqdm
print(tf.__version__)

2.9.0


In [2]:
#FOR ANIMALS
# TRAIN_DIR = "D:/VA/animals_new_dataset"
# FOR NUMBERS
TRAIN_DIR = "E:/VA/standard dataset/Massey categories/"
#TRAIN_DIR = "E:/VA/standard dataset/ArASL_Database_54K_final/"
CATEGORIES = []
for img in tqdm(os.listdir(TRAIN_DIR)):
    CATEGORIES.append(img)
#print(CATEGORIES)
IMG_SIZE=128

100%|███████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 36105.92it/s]


In [3]:
print(CATEGORIES)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
cat_len=len(CATEGORIES)
print(cat_len)

36


In [7]:
# load_path = 'E:/VA/onehandtwohand/224_gray/massey_H/'
load_path = 'E:/VA/onehandtwohand/128/massey_H/'

#load_path = 'E:/VA/onehandtwohand/128/ArASL_H/'
for category in CATEGORIES:
    try: 
        os.makedirs(os.path.join(load_path, category))
    except:
        pass

In [8]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

def landmarks(img,results):
      
# hands
    for hand_landmarks in results.multi_hand_landmarks:

        mp_drawing.draw_landmarks(
          img,
          hand_landmarks,
          mp_hands.HAND_CONNECTIONS,
          mp_drawing_styles.get_default_hand_landmarks_style(),
          mp_drawing_styles.get_default_hand_connections_style())

In [9]:
# for getting just hands and face
training_data = []
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
    
    for category in CATEGORIES:
        path=TRAIN_DIR+"/"+category
        path2=load_path+"/"+category
#         print(path)
        class_num = CATEGORIES.index(category)
        for c_img in tqdm(os.listdir(path)):
            img_array = cv2.imread((path+"/"+c_img))
            
            image_height, image_width, _ = img_array.shape
            # Convert the BGR image to RGB before processing.
            results = hands.process(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
                    
#             
            
            img = np.zeros([image_height, image_width,3],dtype=np.uint8)
            img.fill(255)
            
            if not results.multi_hand_landmarks:  # exclude saving no hands
                continue
            
            landmarks(img,results) # from function
            img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
            #################code for gray only 1 line
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            ################
            for i in range(9):
                cv2.imwrite((path2+"/"+str(i)+c_img),img)
                training_data.append([img, class_num])
            
            # conditions on one_hand
#             if category in one_hand:
#                 if  (results.left_hand_landmarks or results.right_hand_landmarks):   #include saving either hands
#                     landmarks(img,results) # from function
#                     img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
#                     cv2.imwrite((path2+"/"+c_img),img)
#                     training_data.append([img, class_num])
                    
#             # condition on two_hands        
#             if category in two_hand:
#                 if  (results.left_hand_landmarks and results.right_hand_landmarks):   #include both hand
#                     landmarks(img,results) # from function
#                     img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
#                     cv2.imwrite((path2+"/"+c_img),img)
#                     training_data.append([img, class_num])
                
            
import random
random.shuffle(training_data)
# np.save('F:/VA/onehandtwohand/26words_DSLR_H/training1a.npy', training_data)            

100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:02<00:00, 20.40it/s]


In [11]:
X =[]
y =[]
for features, label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
#############gray
#X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
###############
# X = X.astype('float32')
# X /= 255
from keras.utils import np_utils
Y = np_utils.to_categorical(y, cat_len)
np.save(load_path+'X.npy', X)
np.save(load_path+'Y.npy', Y)
print("done")

done


In [12]:
np.save(load_path+'cat_massey.npy',CATEGORIES)

In [ ]:

# exclude situation
one_hand=CATEGORIES

from collections import Counter
two_hand = list((Counter(CATEGORIES)-Counter(one_hand)).elements())
#=['bull','cat','cow','crocodile','dog','elephant','horse','lion','tiger']

print(two_hand)